In [1]:
import pickle
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
import re

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.swnu_network_functions import (
    swnu_network_hyperparameter_search, obtain_SWNUNetwork_input
)

In [4]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [5]:
%run load_sbert-embeddings.py

In [6]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [7]:
x_data = obtain_SWNUNetwork_input(
    method = "gaussian_random_projection",
    dimension= 30,
    df= df_rumours,
    id_column='timeline_id',
    label_column='label',
    embeddings= sbert_embeddings,
    k=5,
    features='time_encoding',
    standardise_method=None,
    include_features_in_path=False,
)

x_data["x_data"]["path"].shape

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


torch.Size([5568, 5, 30])

## SWNU Network

In [8]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
include_features_in_path = True

In [9]:
num_epochs = 100
dimensions = [15]
swnu_hidden_dim_sizes_and_sig_depths = [([12], 3), ([10], 4)]
ffn_hidden_dim_sizes = [[64,64],[128,128],[256,256],[512,512]]
dropout_rates = [0.5, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
bidirectional = True
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5
split_indices = (df_rumours[df_rumours['set']=='train'].index,
                 df_rumours[df_rumours['set']=='dev'].index,
                 df_rumours[df_rumours['set']=='test'].index)

In [10]:
# #dimensionality reduction
# embedding_dim = sbert_embeddings.shape[1]
# dim_reduce_method = ["umap"] #["gaussian_random_projection", "umap"]
# dimensions = [20]#[50,30,15]
# #time features
# features = "time_encoding"
# standardise_method = "standardise"
# include_features_in_path = False
# #SWNU block
# augmentation_tp = "Conv1d"
# hidden_dim_aug = None
# comb_m = "concatenation"
# log_sig = True
# conv_output_channels = [10] #[20, 10, 5]
# log_signature_dimensions_and_sig_depths = [(8, 3)]#[(30, 2), (10, 3), (6, 4)]
# bidirectional = False
# #ffn
# hidden_dim_sizes = [64]#[32,64]
# dropout_rates = [0.2]#[0.5, 0.2, 0.1]
# #overall training
# num_epochs = 100
# batch=64
# patience = 4
# learning_rates = [1e-4] #[1e-3, 1e-4, 5e-4]
# seeds = [0, 1, 12, 123, 1234]
# loss = "focal"
# gamma = 2
# validation_metric = "f1"
# split_indices = (df_rumours[df_rumours['set']=='train'].index,
#                  df_rumours[df_rumours['set']=='dev'].index,
#                  df_rumours[df_rumours['set']=='test'].index)

In [11]:
size=35

## UMAP

In [12]:
swnu_network_umap, best_swnu_network_umap, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=bidirectional,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features, 
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    split_ids=None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices=split_indices,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_umap_focal_2_35.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_umap_focal_2_35_best_model.csv


In [13]:
swnu_network_umap.groupby(["dimensions",
                           "swnu_hidden_dim",
                           "ffn_hidden_dim",
                           "dropout_rate",
                           "learning_rate"]).mean()

/tmp/ipykernel_1597978/373295943.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap.groupby(["dimensions",


loss  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (64, 64)       0.1          0.0001         0.501318   
                                                       0.0005         0.343708   
                                                       0.0010         0.309452   
                                          0.5          0.0001         0.400168   
                                                       0.0005         0.363186   
                                                       0.0010         0.339524   
                           (128, 128)     0.1          0.0001         0.527819   
                                                       0.0005         0.436833   
                                                       0.0010         0.342665   
                                          0.5          0.0001         0.342434   
                                                       0.0005         0.319086   
                                                       0.0010         0.354423   
                           (256, 256)     0.1          0.0001         0.446181   
                                                       0.0005         0.458716   
                                                       0.0010         0.379493   
                                          0.5          0.0001         0.436657   
                                                       0.0005         0.491604   
                                                       0.0010         0.380181   
                           (512, 512)     0.1          0.0001         0.576032   
                                                       0.0005         0.468980   
                                                       0.0010         0.404572   
                                          0.5          0.0001         0.461061   
                                                       0.0005         0.442053   
                                                       0.0010         0.358358   
           (12,)           (64, 64)       0.1          0.0001         0.290559   
                                                       0.0005         0.386859   
                                                       0.0010         0.344226   
                                          0.5          0.0001         0.368774   
                                                       0.0005         0.387525   
                                                       0.0010         0.346880   
                           (128, 128)     0.1          0.0001         0.330256   
                                                       0.0005         0.422683   
                                                       0.0010         0.410526   
                                          0.5          0.0001         0.294226   
                                                       0.0005         0.417337   
                                                       0.0010         0.391361   
                           (256, 256)     0.1          0.0001         0.360631   
                                                       0.0005         0.458986   
                                                       0.0010         0.372693   
                                          0.5          0.0001         0.335267   
                                                       0.0005         0.434963   
                                                       0.0010         0.354384   
                           (512, 512)     0.1          0.0001         0.385747   
                                                       0.0005         0.435122   
                                                       0.0010         0.389019   
                                          0.5          0.0001         0.343538   
                                                       0.0005         0.415543   
                                                       0

In [14]:
best_swnu_network_umap

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,0.333948,0.633937,0.627395,"[0.578021978021978, 0.6767676767676768]",0.656417,"[0.7285318559556787, 0.5843023255813954]",0.641526,"[0.4790528233151184, 0.804]",0.206417,0.661922,...,1,True,focal,2,False,None,Conv1d,None,concatenation,64
0,0.383187,0.608198,0.607233,"[0.5877632898696089, 0.6267029972752044]",0.614031,"[0.6540178571428571, 0.5740432612312812]",0.611849,"[0.5336976320582878, 0.69]",0.186065,0.772242,...,12,True,focal,2,False,None,Conv1d,None,concatenation,64
0,0.346017,0.609152,0.609023,"[0.6019417475728156, 0.6161048689138577]",0.611858,"[0.6444906444906445, 0.579225352112676]",0.611332,"[0.5646630236794171, 0.658]",0.209281,0.693950,...,123,True,focal,2,False,None,Conv1d,None,concatenation,64


0.6369639772772482

In [15]:
best_swnu_network_umap["f1"].mean()

0.6145504264035235

In [16]:
best_swnu_network_umap["precision"].mean()

0.6274352160857555

In [17]:
best_swnu_network_umap["recall"].mean()

0.6215689131754706

In [18]:
np.stack(best_swnu_network_umap["f1_scores"]).mean(axis=0)

array([0.58924234, 0.63985851])

In [19]:
np.stack(best_swnu_network_umap["precision_scores"]).mean(axis=0)

array([0.67568012, 0.57919031])

In [20]:
np.stack(best_swnu_network_umap["recall_scores"]).mean(axis=0)

array([0.52580449, 0.71733333])

## Unidirectional LSTM

In [12]:
swnu_network_umap_uni, best_swnu_network_umap_uni, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=False,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features, 
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    split_ids=None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices=split_indices,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_uni.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_umap_focal_2_35_uni.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_umap_focal_2_35_uni_best_model.csv


In [13]:
swnu_network_umap_uni.groupby(["dimensions",
                               "swnu_hidden_dim",
                               "ffn_hidden_dim",
                               "dropout_rate",
                               "learning_rate"]).mean()

/tmp/ipykernel_1917973/4035633852.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_uni.groupby(["dimensions",


loss  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (64, 64)       0.1          0.0001         0.360039   
                                                       0.0005         0.350700   
                                                       0.0010         0.327866   
                                          0.5          0.0001         0.341996   
                                                       0.0005         0.349566   
                                                       0.0010         0.320726   
                           (128, 128)     0.1          0.0001         0.358317   
                                                       0.0005         0.339834   
                                                       0.0010         0.303645   
                                          0.5          0.0001         0.315919   
                                                       0.0005         0.313627   
                                                       0.0010         0.412201   
                           (256, 256)     0.1          0.0001         0.362953   
                                                       0.0005         0.389788   
                                                       0.0010         0.356578   
                                          0.5          0.0001         0.372266   
                                                       0.0005         0.372617   
                                                       0.0010         0.338369   
                           (512, 512)     0.1          0.0001         0.359364   
                                                       0.0005         0.396318   
                                                       0.0010         0.321813   
                                          0.5          0.0001         0.367016   
                                                       0.0005         0.423696   
                                                       0.0010         0.372871   
           (12,)           (64, 64)       0.1          0.0001         0.295189   
                                                       0.0005         0.375771   
                                                       0.0010         0.404930   
                                          0.5          0.0001         0.244209   
                                                       0.0005         0.336030   
                                                       0.0010         0.323682   
                           (128, 128)     0.1          0.0001         0.323622   
                                                       0.0005         0.367660   
                                                       0.0010         0.356193   
                                          0.5          0.0001         0.323905   
                                                       0.0005         0.428083   
                                                       0.0010         0.324135   
                           (256, 256)     0.1          0.0001         0.326605   
                                                       0.0005         0.463276   
                                                       0.0010         0.560862   
                                          0.5          0.0001         0.334101   
                                                       0.0005         0.463146   
                                                       0.0010         0.351564   
                           (512, 512)     0.1          0.0001         0.361534   
                                                       0.0005         0.545119   
                                                       0.0010         0.470806   
                                          0.5          0.0001         0.436730   
                                                       0.0005         0.506440   
                                                       0

In [14]:
best_swnu_network_umap_uni

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,0.413170,0.605338,0.591806,"[0.5174825174825175, 0.6661290322580644]",0.638277,"[0.7184466019417476, 0.5581081081081081]",0.615186,"[0.40437158469945356, 0.826]",0.230895,0.633452,...,1,False,focal,2,False,None,Conv1d,None,concatenation,64
0,0.571896,0.611058,0.610208,"[0.5920000000000001, 0.628415300546448]",0.616621,"[0.656319290465632, 0.5769230769230769]",0.614581,"[0.5391621129326047, 0.69]",0.235289,0.768683,...,12,False,focal,2,False,None,Conv1d,None,concatenation,64
0,0.650293,0.551001,0.545443,"[0.5957081545064378, 0.49517684887459806]",0.548399,"[0.5633116883116883, 0.5334872979214781]",0.547029,"[0.6320582877959927, 0.462]",0.194062,0.761566,...,123,False,focal,2,False,None,Conv1d,None,concatenation,64


In [15]:
best_swnu_network_umap_uni["f1"].mean()

0.5824853089446776

In [16]:
best_swnu_network_umap_uni["precision"].mean()

0.6010993439452885

In [17]:
best_swnu_network_umap_uni["recall"].mean()

0.5922653309046751

In [18]:
np.stack(best_swnu_network_umap_uni["f1_scores"]).mean(axis=0)

array([0.56839689, 0.59657373])

In [19]:
np.stack(best_swnu_network_umap_uni["precision_scores"]).mean(axis=0)

array([0.64602586, 0.55617283])

In [20]:
np.stack(best_swnu_network_umap_uni["recall_scores"]).mean(axis=0)

array([0.52519733, 0.65933333])

## GRP

In [21]:
swnu_network_grp, best_swnu_network_grp, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=bidirectional,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features, 
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    split_ids=None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices=split_indices,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_grp_focal_{gamma}_{size}.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_grp_focal_2_35.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_grp_focal_2_35_best_model.csv


In [22]:
swnu_network_grp.groupby(["dimensions",
                           "swnu_hidden_dim",
                           "ffn_hidden_dim",
                           "dropout_rate",
                           "learning_rate"]).mean()

/tmp/ipykernel_1917973/1369330714.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_grp.groupby(["dimensions",


loss  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (64, 64)       0.1          0.0001         0.297851   
                                                       0.0005         0.422844   
                                                       0.0010         0.453006   
                                          0.5          0.0001         0.321159   
                                                       0.0005         0.347587   
                                                       0.0010         0.371043   
                           (128, 128)     0.1          0.0001         0.361233   
                                                       0.0005         0.418845   
                                                       0.0010         0.441688   
                                          0.5          0.0001         0.356776   
                                                       0.0005         0.408368   
                                                       0.0010         0.410961   
                           (256, 256)     0.1          0.0001         0.442860   
                                                       0.0005         0.431176   
                                                       0.0010         0.409005   
                                          0.5          0.0001         0.363993   
                                                       0.0005         0.491508   
                                                       0.0010         0.418373   
                           (512, 512)     0.1          0.0001         0.470469   
                                                       0.0005         0.491330   
                                                       0.0010         0.426329   
                                          0.5          0.0001         0.575007   
                                                       0.0005         0.398048   
                                                       0.0010         0.543648   
           (12,)           (64, 64)       0.1          0.0001         0.365415   
                                                       0.0005         0.388164   
                                                       0.0010         0.421529   
                                          0.5          0.0001         0.410850   
                                                       0.0005         0.374655   
                                                       0.0010         0.352747   
                           (128, 128)     0.1          0.0001         0.401290   
                                                       0.0005         0.356991   
                                                       0.0010         0.442066   
                                          0.5          0.0001         0.331082   
                                                       0.0005         0.392198   
                                                       0.0010         0.359655   
                           (256, 256)     0.1          0.0001         0.410096   
                                                       0.0005         0.405284   
                                                       0.0010         0.407670   
                                          0.5          0.0001         0.321966   
                                                       0.0005         0.397480   
                                                       0.0010         0.532577   
                           (512, 512)     0.1          0.0001         0.512911   
                                                       0.0005         0.338731   
                                                       0.0010         0.552831   
                                          0.5          0.0001         0.467744   
                                                       0.0005         0.433282   
                                                       0

In [23]:
best_swnu_network_grp

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,0.381269,0.635844,0.635804,"[0.6319845857418113, 0.6396226415094339]",0.638057,"[0.6707566462167689, 0.6053571428571428]",0.637725,"[0.5974499089253188, 0.678]",0.179601,0.804270,...,1,True,focal,2,False,None,Conv1d,None,concatenation,64
0,0.254553,0.627264,0.605198,"[0.511860174781523, 0.6985350809560523]",0.690937,"[0.8134920634920635, 0.5683814303638645]",0.639703,"[0.37340619307832423, 0.906]",0.243652,0.690391,...,12,True,focal,2,False,None,Conv1d,None,concatenation,64
0,0.434506,0.596759,0.596757,"[0.5975261655566129, 0.5959885386819483]",0.597941,"[0.6254980079681275, 0.5703839122486288]",0.597974,"[0.5719489981785064, 0.624]",0.173473,0.768683,...,123,True,focal,2,False,None,Conv1d,None,concatenation,64


0.6118406155835983

In [24]:
best_swnu_network_grp["f1"].mean()

0.6125861978712303

In [25]:
best_swnu_network_grp["precision"].mean()

0.642311533857766

In [26]:
best_swnu_network_grp["recall"].mean()

0.6251341833636915

In [27]:
np.stack(best_swnu_network_grp["f1_scores"]).mean(axis=0)

array([0.58045698, 0.64471542])

In [28]:
np.stack(best_swnu_network_grp["precision_scores"]).mean(axis=0)

array([0.70324891, 0.58137416])

In [29]:
np.stack(best_swnu_network_grp["recall_scores"]).mean(axis=0)

array([0.51426837, 0.736     ])

## Unidirectional LSTM

In [30]:
swnu_network_grp_uni, best_swnu_network_grp_uni, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=False,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features, 
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    split_ids=None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices=split_indices,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_grp_focal_{gamma}_{size}_uni.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_grp_focal_2_35_uni.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_grp_focal_2_35_uni_best_model.csv


In [31]:
swnu_network_grp_uni.groupby(["dimensions",
                              "swnu_hidden_dim",
                              "ffn_hidden_dim",
                              "dropout_rate",
                              "learning_rate"]).mean()

/tmp/ipykernel_1917973/2827013940.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_grp_uni.groupby(["dimensions",


loss  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (64, 64)       0.1          0.0001         0.335073   
                                                       0.0005         0.388761   
                                                       0.0010         0.518921   
                                          0.5          0.0001         0.344333   
                                                       0.0005         0.367809   
                                                       0.0010         0.391098   
                           (128, 128)     0.1          0.0001         0.330173   
                                                       0.0005         0.339445   
                                                       0.0010         0.378575   
                                          0.5          0.0001         0.398534   
                                                       0.0005         0.354761   
                                                       0.0010         0.391799   
                           (256, 256)     0.1          0.0001         0.482515   
                                                       0.0005         0.520985   
                                                       0.0010         0.696119   
                                          0.5          0.0001         0.336181   
                                                       0.0005         0.372830   
                                                       0.0010         0.348430   
                           (512, 512)     0.1          0.0001         0.434134   
                                                       0.0005         0.514905   
                                                       0.0010         0.591196   
                                          0.5          0.0001         0.342914   
                                                       0.0005         0.557997   
                                                       0.0010         0.386394   
           (12,)           (64, 64)       0.1          0.0001         0.285304   
                                                       0.0005         0.381647   
                                                       0.0010         0.475192   
                                          0.5          0.0001         0.283506   
                                                       0.0005         0.328780   
                                                       0.0010         0.346936   
                           (128, 128)     0.1          0.0001         0.364167   
                                                       0.0005         0.409680   
                                                       0.0010         0.424686   
                                          0.5          0.0001         0.315222   
                                                       0.0005         0.451950   
                                                       0.0010         0.402133   
                           (256, 256)     0.1          0.0001         0.362050   
                                                       0.0005         0.435905   
                                                       0.0010         0.430300   
                                          0.5          0.0001         0.292459   
                                                       0.0005         0.387889   
                                                       0.0010         0.415212   
                           (512, 512)     0.1          0.0001         0.378167   
                                                       0.0005         0.459788   
                                                       0.0010         0.434649   
                                          0.5          0.0001         0.359080   
                                                       0.0005         0.463806   
                                                       0

In [32]:
best_swnu_network_grp_uni

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,0.321927,0.617731,0.617229,"[0.6310947562097515, 0.6033630069238378]",0.617208,"[0.637546468401487, 0.5968688845401174]",0.617386,"[0.6247723132969034, 0.61]",0.184549,0.775801,...,1,False,focal,2,False,None,Conv1d,None,concatenation,64
0,0.447548,0.606292,0.604428,"[0.5772773797338792, 0.631578947368421]",0.614463,"[0.6588785046728972, 0.5700483091787439]",0.610831,"[0.5136612021857924, 0.708]",0.236997,0.722420,...,12,False,focal,2,False,None,Conv1d,None,concatenation,64
0,0.586376,0.589133,0.585807,"[0.5486910994764398, 0.6229221347331583]",0.599487,"[0.645320197044335, 0.5536547433903577]",0.594616,"[0.4772313296903461, 0.712]",0.200912,0.722420,...,123,False,focal,2,False,None,Conv1d,None,concatenation,64


In [33]:
best_swnu_network_grp_uni["f1"].mean()

0.6024878874075813

In [34]:
best_swnu_network_grp_uni["precision"].mean()

0.6103861845379898

In [35]:
best_swnu_network_grp_uni["recall"].mean()

0.6076108075288403

In [36]:
np.stack(best_swnu_network_grp_uni["f1_scores"]).mean(axis=0)

array([0.58568775, 0.61928803])

In [37]:
np.stack(best_swnu_network_grp_uni["precision_scores"]).mean(axis=0)

array([0.64724839, 0.57352398])

In [38]:
np.stack(best_swnu_network_grp_uni["recall_scores"]).mean(axis=0)

array([0.53855495, 0.67666667])